In [1]:
TE = joinpath(tempdir(), "OnePiece.test")

mkpath(TE)

"/var/folders/mg/30npdf0s1z36txb2qvgylrzh0000gn/T/OnePiece.test"

---

In [2]:
pam = dirname(@__DIR__)

pap = joinpath(pam, "Project.toml")

pas = joinpath(pam, "src")

pat = joinpath(pam, "test")

;

In [3]:
using TOML

In [4]:
to = TOML.parse(read(pap, String))

mo = "OnePiece"

@assert to["name"] == splitext(basename(pam))[1] == mo

## Make src/OnePiece.jl 

In [5]:
su_ = [
    "io/table",
    "io/fcs",
    "io/gct",
    "io/gmt",
    #"io/pandas",
    "extension/constant",
    "extension/dict",
    "extension/vector",
    "extension/string",
    "extension/path",
    "extension/dataframe",
    "templating",
    "informatics/tensor",
    "informatics/geometry",
    "informatics/normalization",
    "informatics/statistics",
    "informatics/significance",
    "informatics/information",
    "emoji",
    "figure",
    "tensor_function",
    "feature_by_sample",
    "gene",
    "feature_set_enrichment",
]

;

In [6]:
dis_ = Set(vcat([splitpath(su) for su in su_]...))

di_ = Set(vcat([wa[2] for wa in walkdir(pas)]...))

symdiff(dis_, di_)

Set{String} with 1 element:
  "pandas"

In [7]:
using OrderedCollections

In [8]:
tr = OrderedDict()

tr[mo] = OrderedDict()

cu = tr[mo]

for su in su_

    di_ = splitpath(su)

    n_di = length(di_)

    for (id, di) in enumerate(di_)

        if id < n_di

            if !haskey(cu, di)

                cu[di] = OrderedDict()

            end

            cu = cu[di]

        else

            jl_ = [fi for fi in readdir(joinpath(pas, su)) if occursin(r"\.jl$", fi)]

            cu[di] = [joinpath(su, jl) for jl in jl_]

            cu = tr[mo]

        end

    end

end

In [9]:
function write_line(io, st)

    return write(io, string(st, "\n"))

end

write_line (generic function with 1 method)

In [10]:
function write_branch(io, id, va)

    sp = "  "^id

    if va isa OrderedDict

        for (ke, va2) in va

            write_line(io, string(sp, "module ", ke))

            write_branch(io, id + 1, va2)

            write_line(io, string(sp, "end"))

        end

    else

        for li in va

            write_line(io, string(sp, "include(\"", li, "\")"))

        end

    end


end

write_branch (generic function with 1 method)

In [11]:
open(joinpath(pas, string(mo, ".jl")), "w") do io

    write_branch(io, 0, tr)

end

In [12]:
using OnePiece

┌ Info: Precompiling OnePiece [1fe83854-81c0-42f7-afc9-71ba9af673ca]
└ @ Base loading.jl:1423


## Template

In [13]:
de, id_ = "-"^3, [1, 2, 1]

for (id, su) in enumerate(su_)

    nb = joinpath(pat, string(su, ".ipynb"))

    println("Templating ", nb, " (", id, ")")

    run(`jupyter-nbconvert --clear-output --inplace --log-level 0 $nb`)

    OnePiece.templating.transplant("template.ipynb", nb, de, id_)

end

Templating /Users/kwat/craft/OnePiece.jl/test/io/table.ipynb (1)
Transplanting /Users/kwat/craft/OnePiece.jl/test/io/table.ipynb
Templating /Users/kwat/craft/OnePiece.jl/test/io/fcs.ipynb (2)
Transplanting /Users/kwat/craft/OnePiece.jl/test/io/fcs.ipynb
Templating /Users/kwat/craft/OnePiece.jl/test/io/gct.ipynb (3)
Transplanting /Users/kwat/craft/OnePiece.jl/test/io/gct.ipynb
Templating /Users/kwat/craft/OnePiece.jl/test/io/gmt.ipynb (4)
Transplanting /Users/kwat/craft/OnePiece.jl/test/io/gmt.ipynb
Templating /Users/kwat/craft/OnePiece.jl/test/extension/constant.ipynb (5)
Transplanting /Users/kwat/craft/OnePiece.jl/test/extension/constant.ipynb
Templating /Users/kwat/craft/OnePiece.jl/test/extension/dict.ipynb (6)
Transplanting /Users/kwat/craft/OnePiece.jl/test/extension/dict.ipynb
Templating /Users/kwat/craft/OnePiece.jl/test/extension/vector.ipynb (7)
Transplanting /Users/kwat/craft/OnePiece.jl/test/extension/vector.ipynb
Templating /Users/kwat/craft/OnePiece.jl/test/extension/strin

## Test

In [ ]:
for (id, su) in enumerate(su_)

    nb = joinpath(pat, string(su, ".ipynb"))

    if id < 1

        continue

    end

    println(id)

    println("Running ", nb, " (", id, ")")

    run(
        `jupyter-nbconvert --execute --ExecutePreprocessor.timeout=-1 --clear-output --inplace --log-level 0 $nb`,
    )

end

---

In [ ]:
rm(TE; recursive = true)